In [1]:
import toml
import pathlib

import numpy as np
import torch

import pyworld as pw
import librosa
import cv2

from cmvc import *
from cmvc.utils.data.dataset import PairDataset

In [2]:
import IPython.display as display

In [3]:
dict_toml = toml.load(open('/home/jun/Documents/CMVC/cmvc/config.toml'))

image_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["image"])
voice_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["voice"])

In [90]:
uttr_path = pathlib.Path(dict_toml["path"]["dataset"]["voice"])

In [5]:
dataset = PairDataset(voice_path=voice_path, image_path=image_path, train=True)

In [6]:
eval_data = PairDataset(voice_path=voice_path, image_path=image_path, train=False)

In [51]:
tmp = eval_data.__getitem__(idx=0, k=1)
wav, img = tmp[0], tmp[1]

In [52]:
wav, img

(tensor([[[[0.3439, 0.3390, 0.3373,  ..., 0.3445, 0.3456, 0.3439],
           [2.6578, 2.6508, 2.6484,  ..., 2.6587, 2.6601, 2.6577],
           [2.6576, 2.6507, 2.6482,  ..., 2.6585, 2.6600, 2.6575],
           ...,
           [2.6237, 2.6246, 2.6163,  ..., 2.6216, 2.6224, 2.6214],
           [2.6229, 2.6236, 2.6155,  ..., 2.6202, 2.6210, 2.6201],
           [2.6221, 2.6226, 2.6146,  ..., 2.6189, 2.6196, 2.6188]]]]),
 tensor([[[[0.6902, 0.5725, 0.4118,  ..., 0.1137, 0.1059, 0.1020],
           [0.6784, 0.6196, 0.4510,  ..., 0.1255, 0.1490, 0.1176],
           [0.6627, 0.6902, 0.4824,  ..., 0.1725, 0.1451, 0.1608],
           ...,
           [0.0863, 0.1137, 0.2667,  ..., 0.2510, 0.2510, 0.2392],
           [0.0667, 0.0235, 0.2314,  ..., 0.1843, 0.2706, 0.2471],
           [0.0863, 0.0784, 0.1922,  ..., 0.1333, 0.1059, 0.2510]],
 
          [[0.7412, 0.6314, 0.4902,  ..., 0.1373, 0.1412, 0.1451],
           [0.7255, 0.6745, 0.5294,  ..., 0.1490, 0.1804, 0.1490],
           [0.7059, 0.7

In [53]:
voice_inv = dataset.voice_transform.inv_trans
voice_trans = dataset.voice_transform.__call__
image_inv = dataset.image_transform.inv_trans
image_trans = dataset.image_transform.__call__

In [91]:
uttr_path = pathlib.Path(dict_toml["path"]["dataset"]["voice"])/"vcc2018_database_evaluation/vcc2018_evaluation"/"VCC2SF1"

In [92]:
wav_file = list(uttr_path.iterdir())[0]

In [93]:
wav_file

PosixPath('/mnt/f/dataset/DS_10283_3061/vcc2018_database_evaluation/vcc2018_evaluation/VCC2SF1/30001.wav')

In [106]:
display.Audio(wav_file)

In [121]:
x, fs = librosa.load(wav_file, sr=22050, dtype=np.float64)
#mccs = librosa.feature.mfcc(x, fs, n_mfcc=36)

In [124]:
mfcc = librosa.feature.mfcc(y=x, sr=22050, n_mfcc=36)

In [125]:
aud = librosa.feature.inverse.mfcc_to_audio(mfcc)

In [128]:
sf.write("aud.wav", aud, 22050, "PCM_16")

In [129]:
display.Audio("aud.wav")

In [107]:
aud.shape

(91648,)

In [56]:
#mccs

In [98]:
x.max()

0.394622802734375

In [13]:
#img = image_path / "000085.jpg"

In [14]:
#display.Image(img)

In [57]:
net = Net()

In [58]:
model_dir = pathlib.Path('/home/jun/Documents/CMVC/checkpoint')
checkpoint = torch.load(model_dir/(str(10).zfill(4)+".cpt"))

In [59]:
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [60]:
def eval_net(net, voice, image,
             device="cpu"):
    net.eval()
    
    x = voice.to(device)
    y = image.to(device)
    
    return net.forward(x, y)

In [19]:
"""
x, fs = librosa.load(wav, sr=22050, dtype=np.float64)
mccs = librosa.feature.melspectrogram(y=x, sr=fs, n_mels=36)
"""

'\nx, fs = librosa.load(wav, sr=22050, dtype=np.float64)\nmccs = librosa.feature.melspectrogram(y=x, sr=fs, n_mels=36)\n'

In [20]:
"""
xx = torch.tensor([[voice_trans(mccs)]])
yy = torch.tensor(image_trans(cv2.imread(str(img))))
"""

'\nxx = torch.tensor([[voice_trans(mccs)]])\nyy = torch.tensor(image_trans(cv2.imread(str(img))))\n'

In [62]:
xx = wav
yy = img

In [63]:
print(xx, yy)

tensor([[[[0.3439, 0.3390, 0.3373,  ..., 0.3445, 0.3456, 0.3439],
          [2.6578, 2.6508, 2.6484,  ..., 2.6587, 2.6601, 2.6577],
          [2.6576, 2.6507, 2.6482,  ..., 2.6585, 2.6600, 2.6575],
          ...,
          [2.6237, 2.6246, 2.6163,  ..., 2.6216, 2.6224, 2.6214],
          [2.6229, 2.6236, 2.6155,  ..., 2.6202, 2.6210, 2.6201],
          [2.6221, 2.6226, 2.6146,  ..., 2.6189, 2.6196, 2.6188]]]]) tensor([[[[0.6902, 0.5725, 0.4118,  ..., 0.1137, 0.1059, 0.1020],
          [0.6784, 0.6196, 0.4510,  ..., 0.1255, 0.1490, 0.1176],
          [0.6627, 0.6902, 0.4824,  ..., 0.1725, 0.1451, 0.1608],
          ...,
          [0.0863, 0.1137, 0.2667,  ..., 0.2510, 0.2510, 0.2392],
          [0.0667, 0.0235, 0.2314,  ..., 0.1843, 0.2706, 0.2471],
          [0.0863, 0.0784, 0.1922,  ..., 0.1333, 0.1059, 0.2510]],

         [[0.7412, 0.6314, 0.4902,  ..., 0.1373, 0.1412, 0.1451],
          [0.7255, 0.6745, 0.5294,  ..., 0.1490, 0.1804, 0.1490],
          [0.7059, 0.7412, 0.5490,  ..., 

In [64]:
device = "cpu"
net.to(device)

Net(
  (ue): UttrEncoder(
    (uttr_enc_d1): CBGLayer(
      (conv1): Conv2d(1, 16, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
      (conv2): Conv2d(1, 16, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (uttr_enc_d2): CBGLayer(
      (conv1): Conv2d(16, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (conv2): Conv2d(16, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (uttr_enc_d3): CBGLayer(
      (conv1): Conv2d(32, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (conv2): Conv2d(32, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
      (bn1): BatchN

In [65]:
wav

tensor([[[[0.3439, 0.3390, 0.3373,  ..., 0.3445, 0.3456, 0.3439],
          [2.6578, 2.6508, 2.6484,  ..., 2.6587, 2.6601, 2.6577],
          [2.6576, 2.6507, 2.6482,  ..., 2.6585, 2.6600, 2.6575],
          ...,
          [2.6237, 2.6246, 2.6163,  ..., 2.6216, 2.6224, 2.6214],
          [2.6229, 2.6236, 2.6155,  ..., 2.6202, 2.6210, 2.6201],
          [2.6221, 2.6226, 2.6146,  ..., 2.6189, 2.6196, 2.6188]]]])

In [20]:
voice_inv(xx).min()

TypeError: unsupported operand type(s) for *: 'PosixPath' and 'float'

In [38]:
xx.size()

TypeError: 'int' object is not callable

In [26]:
#net.fe.face_encoder(yy)[0]

In [28]:
#net.fe(yy)

In [66]:
eval_net(net, xx, yy).min()

tensor(0.8626, grad_fn=<MinBackward1>)

In [67]:
conversion_mccs_tensor = eval_data.voice_transform.inv_trans(eval_net(net, xx, yy)).squeeze(0).squeeze(0)

In [68]:
conversion_mccs = conversion_mccs_tensor.detach().numpy().copy()

In [69]:
conversion_mccs.shape

(36, 180)

In [131]:
mccs = xx.squeeze(0).squeeze(0).detach().numpy().copy()

In [80]:
mccs.shape

(36, 180)

In [79]:
conversion_mccs.shape

(36, 180)

In [108]:
conv_audio = librosa.feature.inverse.mfcc_to_audio(conversion_mccs)

In [85]:
conv_audio.max()

0.3138021

In [134]:
aud = librosa.feature.inverse.mfcc_to_audio(mccs)

In [73]:
def mccs_to_audio(mccs):
    mccs = mccs.astype("float32")
    #mccs += mccs.min()
    S = librosa.feature.inverse.mel_to_stft(mccs)
    y = librosa.griffinlim(S, dtype=np.float32)
    
    return y

In [72]:
#y = mccs_to_audio(conversion_mccs)

In [100]:
import soundfile as sf

In [110]:
sf.write("conv.wav", conv_audio, 22050, "PCM_16")

In [111]:
display.Audio("conv.wav")

In [135]:
sf.write("aud.wav", aud, 22050, "PCM_16")

In [136]:
display.Audio("aud.wav")